<a href="https://colab.research.google.com/github/ChairaWandau/MLHomeworks/blob/main/ML_LW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Лабораторная работа №4
Студентка Самсонова Валерия, группа М8О-107М-22

Оптимизировать алгоритмы градиентного спуска с помощью Optuna

##Часть 1
Проведем оптимизацию градиентного спуска на двух функциях оптимизации

Установим optuna

In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 6.7 MB/s 
     |████████████████████████████████| 209 kB 43.7 MB/s 
     |████████████████████████████████| 81 kB 7.1 MB/s 
     |████████████████████████████████| 78 kB 7.3 MB/s 
     |████████████████████████████████| 112 kB 66.4 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 147 kB 67.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=33a4732c937291f369961e3be917a12e2f586f14bd5e500164dbe131ec07e2b7
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


Подключаем необходимые библиотеки

In [52]:
import numpy as np
import optuna
import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection
import logging
import sys
#Графики
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

Записываем функцию Гольдшейна-Прайса

In [16]:
#Функция Гольдшейна-Прайса
def GP_E(x,y):
    return (1+(x+y+1)**2 * (19-14*x+3*x**2-14*y+6*x*y+3*y**2))*(30+(2*x-3*y)**2 * (18-32*x+12*x**2+48*y-36*x*y+27*y**2))
def objective_GP(trial):
    x = trial.suggest_float("x", -2, 2)
    y = trial.suggest_float("y", -2, 2)
    return GP_E(x, y)

Записываем функцию Химмельблау

In [17]:
#Функция Химмельблау
def H_E(x,y):
  return (x**2+y-11)**2+(x+y**2-7)**2
def objective_H(trial):
    x = trial.suggest_float("x", -5, 5)
    y = trial.suggest_float("y", -5, 5)
    return H_E(x, y)

In [18]:
study_GP = optuna.create_study()
study_GP.optimize(objective_GP, n_trials=100)

study_H = optuna.create_study()
study_H.optimize(objective_H, n_trials=100)

[I 2022-11-13 17:24:53,167] A new study created in memory with name: no-name-7cd432f2-f1a7-40d8-86d6-466788925dda
[I 2022-11-13 17:24:53,172] Trial 0 finished with value: 8493.95489474492 and parameters: {'x': 0.9084521201629907, 'y': -1.2950879013889605}. Best is trial 0 with value: 8493.95489474492.
[I 2022-11-13 17:24:53,179] Trial 1 finished with value: 246789.64161973243 and parameters: {'x': -0.5188203260984414, 'y': 1.615015166605919}. Best is trial 0 with value: 8493.95489474492.
[I 2022-11-13 17:24:53,184] Trial 2 finished with value: 105469.7007862363 and parameters: {'x': -0.16436498846361358, 'y': 1.4170148190083496}. Best is trial 0 with value: 8493.95489474492.
[I 2022-11-13 17:24:53,188] Trial 3 finished with value: 166.31787068488995 and parameters: {'x': -0.4785289384293847, 'y': -0.8433835216453844}. Best is trial 3 with value: 166.31787068488995.
[I 2022-11-13 17:24:53,193] Trial 4 finished with value: 786.8165804903496 and parameters: {'x': 0.12220896836216655, 'y':

Выводим результат

In [20]:
best_params_GP = study_GP.best_params
found_x = best_params_GP["x"]
found_y = best_params_GP["y"]
print("Функция Гольдшейна-Прайса:")
print(f"x: {found_x}, y: {found_y}, E: {GP_E(found_x, found_y)}")
best_params_H = study_H.best_params
found_x = best_params_H["x"]
found_y = best_params_H["y"]
print("Функция Химмельблау:")
print(f"x: {found_x}, y: {found_y}, E: {H_E(found_x, found_y)}")

Функция Гольдшейна-Прайса:
x: -0.016585135340439733, y: -0.956219127535605, E: 3.9894198326836223
Функция Химмельблау:
x: -2.74501531414102, y: 3.004744729792382, E: 0.7251417697341881


##Часть 2
Проведем оптимизацию градиентного спуска на датасете Diabetes

In [44]:
def objective(trial):
    BC = sklearn.datasets.load_diabetes()
    classes = list(set(BC.target))
    train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(
        BC.data, BC.target, test_size=0.25, random_state=0)
    alpha = trial.suggest_float("alpha", 1e-5, 1e-1, log=True)
    clf = sklearn.linear_model.SGDClassifier(alpha=alpha)
    for step in range(100):
        clf.partial_fit(train_x, train_y, classes=classes)
        intermediate_value = 1.0 - clf.score(valid_x, valid_y)
        trial.report(intermediate_value, step)
        if trial.should_prune():
            raise optuna.TrialPruned()
    return 1.0 - clf.score(valid_x, valid_y)

In [45]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20)

[I 2022-11-13 18:07:13,844] A new study created in memory with name: no-name-3ae68bbf-8d38-4f38-91ae-df88fad04afc


A new study created in memory with name: no-name-3ae68bbf-8d38-4f38-91ae-df88fad04afc
A new study created in memory with name: no-name-3ae68bbf-8d38-4f38-91ae-df88fad04afc
A new study created in memory with name: no-name-3ae68bbf-8d38-4f38-91ae-df88fad04afc
A new study created in memory with name: no-name-3ae68bbf-8d38-4f38-91ae-df88fad04afc


[I 2022-11-13 18:07:22,159] Trial 0 finished with value: 1.0 and parameters: {'alpha': 0.00017496238466290127}. Best is trial 0 with value: 1.0.


Trial 0 finished with value: 1.0 and parameters: {'alpha': 0.00017496238466290127}. Best is trial 0 with value: 1.0.
Trial 0 finished with value: 1.0 and parameters: {'alpha': 0.00017496238466290127}. Best is trial 0 with value: 1.0.
Trial 0 finished with value: 1.0 and parameters: {'alpha': 0.00017496238466290127}. Best is trial 0 with value: 1.0.
Trial 0 finished with value: 1.0 and parameters: {'alpha': 0.00017496238466290127}. Best is trial 0 with value: 1.0.


[I 2022-11-13 18:07:30,369] Trial 1 finished with value: 1.0 and parameters: {'alpha': 0.0016142203351917478}. Best is trial 0 with value: 1.0.


...

Trial 17 finished with value: 1.0 and parameters: {'alpha': 0.009919987407471208}. Best is trial 2 with value: 0.990990990990991.
Trial 17 finished with value: 1.0 and parameters: {'alpha': 0.009919987407471208}. Best is trial 2 with value: 0.990990990990991.
Trial 17 finished with value: 1.0 and parameters: {'alpha': 0.009919987407471208}. Best is trial 2 with value: 0.990990990990991.
Trial 17 finished with value: 1.0 and parameters: {'alpha': 0.009919987407471208}. Best is trial 2 with value: 0.990990990990991.


[I 2022-11-13 18:08:45,646] Trial 18 finished with value: 1.0 and parameters: {'alpha': 0.00041391643021731144}. Best is trial 2 with value: 0.990990990990991.


Trial 18 finished with value: 1.0 and parameters: {'alpha': 0.00041391643021731144}. Best is trial 2 with value: 0.990990990990991.
Trial 18 finished with value: 1.0 and parameters: {'alpha': 0.00041391643021731144}. Best is trial 2 with value: 0.990990990990991.
Trial 18 finished with value: 1.0 and parameters: {'alpha': 0.00041391643021731144}. Best is trial 2 with value: 0.990990990990991.
Trial 18 finished with value: 1.0 and parameters: {'alpha': 0.00041391643021731144}. Best is trial 2 with value: 0.990990990990991.


[I 2022-11-13 18:08:54,571] Trial 19 finished with value: 1.0 and parameters: {'alpha': 0.003447198232149919}. Best is trial 2 with value: 0.990990990990991.


Trial 19 finished with value: 1.0 and parameters: {'alpha': 0.003447198232149919}. Best is trial 2 with value: 0.990990990990991.
Trial 19 finished with value: 1.0 and parameters: {'alpha': 0.003447198232149919}. Best is trial 2 with value: 0.990990990990991.
Trial 19 finished with value: 1.0 and parameters: {'alpha': 0.003447198232149919}. Best is trial 2 with value: 0.990990990990991.
Trial 19 finished with value: 1.0 and parameters: {'alpha': 0.003447198232149919}. Best is trial 2 with value: 0.990990990990991.


In [46]:
plot_optimization_history(study)

In [47]:
plot_intermediate_values(study)

In [48]:
plot_parallel_coordinate(study)

In [49]:
plot_slice(study)

In [50]:
plot_param_importances(study)

In [51]:
plot_edf(study)